In [ ]:
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
from openai import OpenAI


In [ ]:
import os
import pandas as pd
files = os.listdir()
# for i in files:
#     if i.endswith('.xlsx'):
#         print(i)
import json
filepath = 'Flow chart english.xlsx'
os.path.exists(filepath)

# Load Excel (adjust path and sheet name as needed)
df = pd.read_excel(filepath, header=9)
columns_to_drop = ['Process Symbols','Improve content', 'Inspection Problem Description','Improvement goals.1', 
                   'Current Level', 'Improvement goals.2', 'Unnamed: 25', 'Unnamed: 26', 'Expected', 'actual', 'BU', 'Manufacturing CoE-IE',
                   'Number of operators', 'Average working hours at workstations (Sec.)','Unnamed: 34','Unnamed: 35',
                   'Assignment content.1','Single time','Number of jobs','Total working hours', ' Saturation']
df = df.drop(columns_to_drop, axis=1)

print(len(df))
# df.head()
# print(df.to_string(index=False, max_cols=None))

In [ ]:
df = df.rename(columns={'name': 'Product Number', 
                        'Serial number': 'Workstation number', 
                        'name.1': 'Workstation name',
                        'Serial number.1': 'Serial number',
                        'Unnamed: 10': 'Number of operators',
                        'Unnamed: 11': 'Average Working hours (Sec.)',
                        'Unnamed: 12': 'Current manual labor saturation',
                        'Process': 'Value Judgment Process',
                        'unit': 'Value Judgment unit',
                        })
print(df.to_string(index=False, max_cols=None))

In [ ]:
df['Workstation number'] = df['Workstation number'].fillna(method='ffill')
df['Workstation name'] = df['Workstation name'].fillna(method='ffill')
df['Machine (Sec.)'] = df['Machine (Sec.)'].fillna(0)
df['Number of products'] = df['Number of products'].fillna(method='ffill')
df['Working hours (Sec.)'] = df['Working hours (Sec.)'].fillna(method='ffill')
df['Number of operators'] = df['Number of operators'].fillna(method='ffill')
df['Average Working hours (Sec.)'] = df['Average Working hours (Sec.)'].fillna(method='ffill')
df['Current manual labor saturation'] = df['Current manual labor saturation'].fillna(method='ffill')
df['Value Judgment Process'] = df['Value Judgment Process'].fillna(method='ffill')
df['ECRS'] = df['ECRS'].fillna(method='ffill')

In [ ]:
product_number = '3S6A013-00518-7F'
edition = 'B'
updated  = 263822
approved_by = 'approved by'
reviewer = 'reviewed by'
prepared_by = 'prepared by'
responsible_person = 'responsible person'

# Apply header metadata to all rows
df["Product Number"] = product_number
df["Edition"] = edition
df["Updated"] = updated
df["Approved by"] = approved_by
df["Reviewer"] = reviewer
df["Prepared by"] = prepared_by
df["Responsible Person"] = responsible_person

In [ ]:
df.dropna(subset=['Serial number'], inplace=True)
print(df.to_string(index=False, max_cols=None))

In [ ]:
df['ECRS'] = df['ECRS'].fillna('N/A')

In [ ]:
df.columns
mapping_action_item_classification = {'→': 'Transport', '○': 'Operation', '□': 'Inspect'}
mapping_current_action_level = {'M1': 'Manual', 'M2': 'Semi automation', 'M3': 'Automation', 'M4': 'Advanced automation'}
df['Action Item Classification'] = df['Action Item Classification'].replace(mapping_action_item_classification)
df['Current Action Level'] = df['Current Action Level'].replace(mapping_current_action_level)

In [ ]:
df['Workstation number'] = df['Workstation number'].astype('Int64')
df['Serial number'] = df['Serial number'].astype('Int64')
df['Number of products'] = df['Number of products'].astype('Int64')
df['Number of operators'] = df['Number of operators'].astype('Int64')

In [ ]:
print(df.head(len(df)).to_string(index=False, max_cols=None))

In [ ]:
# # Initialize list for documents
# documents = []

# # Group by Segment
# for segment in df["Workstation number"].unique():
#     segment_df = df[df["Workstation number"] == segment].copy()
    
#     # Extract process-level data
#     workstation_number = segment_df["Workstation number"].iloc[0]
#     workstation_name = segment_df["Workstation name"].iloc[0]
#     number_of_products = segment_df["Number of products"].iloc[0]
#     working_hours_sec = segment_df["Working hours (Sec.)"].iloc[0]
#     number_of_operators = segment_df["Number of operators"].iloc[0]
#     average_working_hours_sec = segment_df["Average Working hours (Sec.)"].iloc[0]
#     current_manual_labor_saturation = segment_df["Current manual labor saturation"].iloc[0]
#     value_judgment_process = segment_df["Value Judgment Process"].iloc[0]

#     # Extract operations
#     operations = []
#     for _, row in segment_df.iterrows():
#         operations.append({
#             "serial_number": row["Serial number"],
#             "description": row["Assignment content"],
#             "manual_sec": row["Manual (Sec.)"],
#             "machine_sec": row["Machine (Sec.)"],
#             "value_judgment_unit": row["Value Judgment unit"],
#             "action_item_classification": row["Action Item Classification"],
#             "current_action_level": row["Current Action Level"]
#         })
#         print(row["Current Action Level"])

In [ ]:
df.columns

In [ ]:
# texts = []
# doc_ids = []
# metadatas = []
# for i, doc in enumerate(documents):
#     # Workstation document
#     workstation_text = "; ".join([
#         f"Workstation Number: {doc['Workstation number']}",
#         f"Workstation Name: {doc['Workstation name']}",
#         f"Number of Products: {doc['Number of products']}",
#         f"Working hours (Sec.): {doc['Working hours (Sec.)']}",
#         f"Number of Operators: {doc['Number of operators']}",
#         f"Cycle Time (Sec.): {doc['Cycle Time (Sec.)']}",
#         f"Job saturation (%): {doc['Job saturation (%)']}",
#         f"Value Judgment Process: {doc['Value Judgment Process']}",
#         f"Improvement Goals: {doc['improvement items']['goals'] if doc['improvement items']['goals'] != '/' else 'No improvements listed'}"
#     ])
#     texts.append(workstation_text)
#     doc_ids.append(f"ws_{i}")
#     metadatas.append({
#         "product_number": doc["product_number"],
#         "workstation_number": str(doc["Workstation number"]),
#         "workstation_name": doc["Workstation name"],
#         "type": "workstation"
#     })
    
#     # Operation documents
#     for j, op in enumerate(doc["operations"]):
#         op_text = "; ".join([
#             f"Workstation Number: {doc['Workstation number']}",
#             f"Workstation Name: {doc['Workstation name']}",
#             f"Serial Number: {op['serial_number']}",
#             f"Description: {op['description']}",
#             f"Manual (sec): {op['manual_sec']}sec",
#             f"Machine (sec): {op['machine_sec']}sec",
#             f"Value Judgment Unit: {op['value_judgment_unit']}",
#             f"Action Item Classification: {op['action_item_classification']}",
#             f"Current Action Level: {op['current_action_level']}"
#         ])
#         texts.append(op_text)
#         doc_ids.append(f"op_{i}_{j}")
#         metadatas.append({
#             "product_number": doc["product_number"],
#             "workstation_number": str(doc["Workstation number"]),
#             "workstation_name": doc["Workstation name"],
#             "serial_number": str(op["serial_number"]),
#             "type": "operation"
#         })

In [ ]:
print(len(texts))
print(len(doc_ids))
print(len(metadatas))
doc_ids
texts[1]

In [ ]:
# Initialize list for documents
documents = []

# Group by Segment
for segment in df["Workstation number"].unique():
    segment_df = df[df["Workstation number"] == segment].copy()
    
    # Extract workstation level data
    workstation_number = segment_df["Workstation number"].iloc[0]
    workstation_name = segment_df["Workstation name"].iloc[0]
    number_of_products = segment_df["Number of products"].iloc[0]
    working_hours_sec = segment_df["Working hours (Sec.)"].iloc[0]
    number_of_operators = segment_df["Number of operators"].iloc[0]
    average_working_hours_sec = segment_df["Average Working hours (Sec.)"].iloc[0]
    current_manual_labor_saturation = segment_df["Current manual labor saturation"].iloc[0]
    value_judgment_process = segment_df["Value Judgment Process"].iloc[0]

    # Extract operation level data
    operations = []
    for _, row in segment_df.iterrows():
        operations.append({
            "serial_number": row["Serial number"],
            "description": row["Assignment content"],
            "manual_sec": row["Manual (Sec.)"],
            "machine_sec": row["Machine (Sec.)"],
            "value_judgment_unit": row["Value Judgment unit"],
            "action_item_classification": row["Action Item Classification"],
            "current_action_level": row["Current Action Level"]
        })
        
    # Extract improvements
    improvements = {
        "problem_description": segment_df["Problem Description"].iloc[0],
        "goals": segment_df["Improvement goals"].iloc[0],
        "ecrs": segment_df["ECRS"].iloc[0],
    }
    
    # Metadata
    metadata = {
        "edition": segment_df["Edition"].iloc[0],
        "updated": segment_df["Updated"].iloc[0],
        "approved_by": segment_df["Approved by"].iloc[0],
        "reviewer": segment_df["Reviewer"].iloc[0],
        "prepared_by": segment_df["Prepared by"].iloc[0],
        "responsible_person": segment_df["Responsible Person"].iloc[0]
    }


# Create document
    document = {
        "product_number": segment_df["Product Number"].iloc[0],
        "Workstation number": workstation_number,
        "Workstation name": workstation_name,
        "Number of products": number_of_products,
        "Working hours (Sec.)": working_hours_sec,
        "Number of operators": number_of_operators,
        "Cycle Time (Sec.)": average_working_hours_sec,
        "Job saturation (%)": current_manual_labor_saturation,
        "Value Judgment Process": value_judgment_process,
        "operations": operations,
        "improvement items": improvements,
        "metadata": metadata
    }
    documents.append(document)        

In [ ]:
documents[0]

In [ ]:
# Saving the data in JSON format

import json
import numpy as np

def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert NumPy arrays to lists
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(elem) for elem in obj]
    return obj

# Assuming 'documents' is your data structure
serializable_documents = convert_to_serializable(documents)

with open("structured_data_no_metrics.json", "w") as f:
    json.dump(serializable_documents, f, indent=2)

In [ ]:
import json
with open("structured_data_no_metrics.json", "r") as f:
    loaded_documents = json.load(f)

In [ ]:
# loaded_documents


In [ ]:
texts = []
doc_ids = []
metadatas = []
for i, doc in enumerate(loaded_documents):
    # Combine fields into a single text string
    workstation_text = ", ".join([
        f"Workstation Number: {doc['Workstation number']}, "
        f"Workstation Name: {doc['Workstation name']}, "
        f"Number of Products: {doc['Number of products']}, "
        f"Working hours (Sec.): {doc['Working hours (Sec.)']}, "
        f"Number of Operators: {doc['Number of operators']}, "
        f"Cycle Time (Sec.): {doc['Cycle Time (Sec.)']}, "
        f"Job saturation (%): {doc['Job saturation (%)']}, "
        f"Value Judgment Process: {doc['Value Judgment Process']}, "])
    
    ops_text = ", ".join([
        f"Serial Number: {op['serial_number']}, " 
        f"Description: {op['description']}, " 
        f"Manual (sec): {op['manual_sec']}sec, " 
        f"Machine (sec): {op['machine_sec']}sec, " 
        f"Value Judgment Unit: {op['value_judgment_unit']}, " 
        f"Action Item Classification: {op['action_item_classification']}, " 
        f"Current Action Level: {op['current_action_level']}"
        for op in doc["operations"]])
    text = f"Workstation: {workstation_text}\nOperations: {ops_text}\nImprovements: {doc['improvement items']['goals'] if doc['improvement items']['goals'] != '/' else 'No improvements listed'}"
    # text = (
    #     f"Workstation: {workstation_text}.\n
    #         {ops_text}. "
    #     f"{doc['improvement items']['goals'] if doc['improvement items']['goals'] != '/' else 'No improvements listed'}."

    # )
    texts.append(text)
    doc_ids.append(str(i))
    metadatas.append({
        "product_number": doc["product_number"],
        "workstation_number": str(doc["Workstation number"]),
        "workstation_name": doc["Workstation name"]
    })

In [ ]:
print(texts[0])

In [ ]:
import textwrap
print(len(doc_ids))
print(len(texts))
print(len(metadatas))

In [ ]:
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")
all_embeddings = embed_model.embed_documents(texts)

In [ ]:
import textwrap
print(len(doc_ids))
print(len(metadatas))
print(len(texts))

print(len(all_embeddings), len(all_embeddings[0]))


In [ ]:
print(texts[-1])

In [ ]:
import logging
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
import chromadb
# Initialize Chroma
## Create a collection for data injection

# Initialize Chroma
collection_name = "improvement_suggestions"
chroma_client = chromadb.PersistentClient(path="./chroma_db_6")

try:
    collection = chroma_client.create_collection(name=collection_name)
    logger.info(f"Created new collection: {collection_name}")
except Exception as e:
    if "already exists" in str(e).lower():
        collection = chroma_client.get_collection(name=collection_name)
        logger.info(f"Collection '{collection_name}' already exists. Using existing collection.")
    else:
        logger.error(f"Failed to create or access collection: {str(e)}")
        raise

In [ ]:
initial_count = collection.count()
print(f'Initial count#: {initial_count}')

In [ ]:
# chroma_client.delete_collection(name=collection_name)
# collection.count()

In [ ]:
collection.add(ids = doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)
final_count = collection.count()
print(f'Final count#: {final_count}')

In [ ]:
# Convert the text back to the JSON format to make the LLM life easy

def structure_document(text: str) -> str:
    """Convert comma-separated document to JSON."""
    fields = {}
    pairs = text.split(", ")
    for pair in pairs:
        if ":" in pair:
            key, value = pair.split(": ", 1)
            fields[key.strip()] = value.strip()
    return json.dumps(fields)

# During ingestion
structured_texts = [structure_document(text) for text in texts]

In [ ]:
collection.add(ids = doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)

In [ ]:
index = 8
print(doc_ids[index])
texts[index]

RAG

In [ ]:
## GROK Suggestions
import logging
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
def validate_collection_inputs(ids, embeddings, documents, metadatas):
    if not all([ids, embeddings, documents]):  # Check for empty inputs
        raise ValueError("IDs, embeddings, and documents must not be empty")
    if len(ids) != len(embeddings) or len(ids) != len(documents):
        raise ValueError("IDs, embeddings, and documents must have the same length")
    if metadatas and len(metadatas) != len(ids):
        raise ValueError("Metadatas length must match IDs length if provided")

try:
    validate_collection_inputs(doc_ids, all_embeddings, texts, metadatas)
    collection.add(ids=doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)
    logger.info("Successfully added data to collection")
except Exception as e:
    logger.error(f"Failed to add data to collection: {str(e)}")
    raise

In [ ]:
template ="""You will recieve the user's question along with the search results of that question \
over a database. Give the user the proper answer.
User's question: {query_texts} 
Search results: {results}
When a question is involved regarding an operation, please fully mention that in your answer.
"""

In [ ]:
template = """You are an expert Industrial Engineering Agent designed to provide accurate, detailed, and practical answers to questions in the field of \
    Industrial Engineering. Please provide the answer based on user query and search results.

**Input:**
- User's question: {query_texts}
- Search results: {results}

**Instructions:**
1. For operations-related questions, include:
   - A definition of the operation.
   - The steps or components involved.
   - Its application or significance in an Industrial Engineering context.
2. If applicable, suggest practical tools, techniques, or methodologies (e.g., Lean, Six Sigma, simulation software) to address the question.
3. If clarification is needed, politely ask the user for additional details to refine the response.
4. If improvement suggestions relevant questions are asked, please answer them in terms of Eliminate, Combine, Rearrange, and Simpify (ECRS) at operation level, include:
    - please mention potential benefit quantitatively 
    - While refering an operation please outline it's full name os user knows which operation you're talking about.
    - and also the challenges to implement them.

**Output:**
A well-structured answer addressing the user's question, grounded in Industrial Engineering principles, with clear explanations of any operations involved."""

In [ ]:

# import chromadb
# # Initialize Chroma
# ## Create a collection for data injection
# collection_name = "improvement_suggestions"
# chroma_client = chromadb.PersistentClient(path="./chroma_db_5")

# try:
#     collection = chroma_client.create_collection(name=collection_name)
# except Exception as e:
#     if "already exists" in str(e):
#         # If the collection already exists, use the existing one
#         collection = chroma_client.get_collection(name=collection_name)
#         print(f"Collection '{collection_name}' already exists. Using the existing collection.")
#     else:
#         # Handle other exceptions
#         raise e


# collection.add(ids = doc_ids, embeddings=all_embeddings, documents=texts, metadatas=metadatas)


from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import textwrap
llm = ChatOpenAI(temperature = 0, model = "gpt-4o",)


# template ="""You will recieve the user's question along with the search results of that question \
# over a database. Give the user the proper answer.
# User's question: {query_texts} 
# Search results: {results}
# When a question is involved regarding an operation, please fully mention that in your answer.
# """
prompt = PromptTemplate.from_template(template=template)
chain = prompt | llm | StrOutputParser()

In [ ]:
The workstations of this production line are:
1. Solder ring assembly & press-in (Workstation Number: 10)
2. Long and short PIN test (Workstation Number: 15)
3. Full inspection & TR packaging (Workstation Number: 17)
4. Coplanarity test 1 (Workstation Number: 12)
5. 0/S Test (Workstation Number: 8)
6. Internal GAP inspection (CCD inspection) (Workstation Number: 13)
7. Hi-pottest&Position detection (Workstation Number: 9)
8. Coplanarity test (retest) (Workstation Number: 14)

In [ ]:
query_texts = "Please analyze the workstation 'Coplanarity test (retest)' and provide improvement suggestions?"
query_embeddings = embed_model.embed_query(query_texts)
# Load the chromadb collection for vector search
vectordb = chroma_client.get_collection(name="improvement_suggestions")
# Retrieve relevant chunk
results = vectordb.query(
    query_embeddings = query_embeddings,
    # n_results=1 #top_k
)
response = chain.invoke({'query_texts': query_texts, 'results': results})
print(response)

In [ ]:
import textwrap
wrapped_text = textwrap.wrap(response[:200], width=100)
wrapped_text

In [ ]:
for key, value in results.items():
    print(key, value)

In [ ]:
len(structured_texts[0])
len(structured_texts[1])

In [ ]:
structured_texts[1]